## Método REINFORCE
El método **REINFORCE**, es una técnica de aprendizaje por refuerzo *policy-based*, esto significa que se entrena una red neuronal para que nos indique cuál puede ser la siguiente acción a realizar, a diferencia de DQN (de la práctica anterior), donde se busca generar los valores de las acciones. En resumen, este método recibe el estado del entorno y retorna directamente la probabilidad de cada acción, por lo que, lo que se busca en el entrenamiento, es hacer que se más probable seleccionar la acción óptima en cada estado, o sea la política óptima.

Características:
- No se usan valores de acción o estado.
- Es adecuado para espacios de acción continua (en general todos los policy-based).
- Es un método *on-policy*, pues los datos del entorno se obtienen siguiendo la misma política que se intenta optimizar.
- Se basa en trayectorias ($\tau$) en lugar de episodios.

Ventajas:
- No es necesario preocuparse por idear una estrategia de exploración del entorno como *epsilon-greedy*.
- No es necesario usar 'trucos' tales como *experience replay* o *target network*.

Desventajas:
- Suele ser menos eficiente en cuanto a muestras y requiere una mayor interacción con el entorno, pues no se puede beneficiar de datos antiguos. (en general todos los policy based)

In [58]:
import numpy as np
import torch
import gymnasium as gym
from matplotlib import pyplot as plt

Crear entorno

In [59]:
env = gym.make('CartPole-v1', render_mode="rgb_array")

Crear red neuronal

In [60]:
obs_size = env.observation_space.shape[0]   # Entrada de la red
n_actions = env.action_space.n              # Salida de la red
HIDDEN_SIZE = 256

model = torch.nn.Sequential(
    torch.nn.Linear(obs_size, HIDDEN_SIZE),
    torch.nn.ReLU(),
    torch.nn.Linear(HIDDEN_SIZE, n_actions),
    torch.nn.Softmax(dim=0) 
)

Hiperparámetros de la red y del entrenamiento

In [61]:
# Hiperparámetros de la red
LEARNING_RATE = 0.003
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Hiperparámetros del algoritmo
HORIZON = 500
MAX_TRAJECTORIES = 500
GAMMA = 0.99

score = []

#### Funciones

In [62]:
# Función para generar una trayectoria
def GenerarTrayectoria(model, estado_actual, horizonte):
    done = False
    transiciones = []
    
    for t in range(horizonte):
        prob_acciones = model(torch.from_numpy(estado_actual).float())         # Se obtienen las probabilidades de las acciones
        accion = np.random.choice(np.array([0,1]), p=prob_acciones.data.numpy()) # Se escoge una acción pero con las probabilidades de la red
        estado_anterior = estado_actual
        estado_actual, recompensa, done, trunc, info = env.step(accion)        # Se ejecuta la acción
        transiciones.append((estado_anterior, accion, t+1))
        if done is True:
            break
    
    return transiciones

# Función para calcular el retorno de una trayectoria
def Rt(batch_recompensas, transiciones, GAMMA):
    batch_Gvals = []
    
    for n in range(len(transiciones)):
        new_Gval = 0
        power = 0
        for m in range(n, len(transiciones)):
            new_Gval = new_Gval + ((GAMMA**power)*batch_recompensas[m]).numpy()
            power += 1
        batch_Gvals.append(new_Gval)
    batch_retornos_estimados = torch.tensor(batch_Gvals)
    batch_retornos_estimados /= batch_retornos_estimados.max()     # Se normalizan los retornos para que no haya problemas de estabilidad numérica en la red
    
    return batch_retornos_estimados   

#### Entrenamiento

El método REINFORCE es un algoritmo *policy gradient*, esto significa que actualiza una red neuronal usando el **ascenso de gradiente**. Los pasos a seguir junto con sus ecuaciones se muestran a continuación:
1. Se usa la política $\pi_{\theta}$ para generar una trayectoria (recordar que la política viene dada por la red neuronal $\theta$, por eso se escribe $\pi_{\theta}$).
2. Se estiman las recompensas en la trayectoria $R(\tau) = (G_{0}, G_{1},..., G_{H})$. Cada $G_{k}$ se calcula con la fórmula: $G_{k}=\sum_{i=k+1}^{H+1} \gamma^{i-k-1}r_{i}$
3. Se estima el gradiente usando $\nabla_{\theta} = \sum_{t=0}^{H} \nabla_{\theta}log\pi_{\theta}(a_{t}|s_{t})G_{t}$. Se observa que se multiplican las probabilidades de cada acción en un estado $\pi_{\theta}(a_{t}|s_{t})$ con su retorno esperado $G_{t}$
4. Se actualizan los parámetros de la red neuronal usando: $\theta = \theta + \alpha \nabla_{\theta}U(\theta)$

In [63]:
for trajectory in range(MAX_TRAJECTORIES):
    estado_actual = env.reset()[0]
    done = False
    
    # Generar una trayectoria
    transiciones = GenerarTrayectoria(model, estado_actual, HORIZON)
    score.append(len(transiciones))                                                   # Se guarda el score de la trayectoria
    batch_recompensas = torch.Tensor([r for (s,a,r) in transiciones]).flip(dims=(0,)) # Se crea un tensor con las recompensas de la trayectoria
    
    # Se calculan el retorno de la trayectoria R(t) = (G0,G1,...,GH)
    batch_retornos_estimados = Rt(batch_recompensas, transiciones, GAMMA)
    
    # Batch de estados y acciones
    batch_estados = torch.Tensor([s for (s,a,r) in transiciones])
    batch_acciones = torch.Tensor([a for (s,a,r) in transiciones])
    
    # Se predice la probabilidad de las acciones con la red neuronal
    predicted_batch = model(batch_estados)
    batch_prob = predicted_batch.gather(dim=1, index=batch_acciones.long().view(-1,1)).squeeze() # Se seleccionan las probabilidades de las acciones que se tomaron antes
    
    # Se calcula la pérdida y se actualizan los pesos
    loss = -torch.sum(torch.log(batch_prob)*batch_retornos_estimados)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if trajectory % 50 == 0 and trajectory>0:
        print('Trajectory {}\tAverage Score: {:.2f}'.format(trajectory, np.mean(score[-50:-1])))

C:\Users\USER\AppData\Local\Temp\ipykernel_12664\1884484331.py:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:248.)
  batch_estados = torch.Tensor([s for (s,a,r) in transiciones])


Trajectory 50	Average Score: 47.92
Trajectory 100	Average Score: 67.35
Trajectory 150	Average Score: 97.61
Trajectory 200	Average Score: 151.86
Trajectory 250	Average Score: 124.47
Trajectory 300	Average Score: 224.24
Trajectory 350	Average Score: 352.14
Trajectory 400	Average Score: 434.43
Trajectory 450	Average Score: 383.27


In [64]:
# Guardar el modelo
torch.save(model.state_dict(), "Practica8_REINFORCE_Net.dat")